In [ ]:
%AddDeps mysql mysql-connector-java 5.1.12

In [ ]:
import java.sql.{Connection, DriverManager, ResultSet}
import scala.util.{Try, Failure}

val url = "jdbc:mysql://mysql:3306/db"
val username = "user"
val pass = "pass"
val driver = "com.mysql.jdbc.Driver"

def getConnection(): Option[Connection] = {
    Try {
        Class.forName(driver).newInstance()
        DriverManager.getConnection(url, username, pass)
    } recoverWith { case e =>
        println("ERROR while establishing mysql connection")
        e.printStackTrace()
        Failure(e)
    } toOption
}

for {conn <- getConnection()} {
    conn.createStatement.executeUpdate("""CREATE TABLE IF NOT EXISTS users (
        id int(11) NOT NULL AUTO_INCREMENT,
        email varchar(255) NOT NULL, 
        name varchar(255) NOT NULL,
        PRIMARY KEY (id) 
        ) AUTO_INCREMENT=1 ;""")

    val email = "test@mail.com"
    val name = "name"
    conn.createStatement.executeUpdate(s"INSERT INTO users VALUES (default, '$email', '$name')")

    val rs = conn.createStatement.executeQuery(s"SELECT id, name FROM users WHERE email='$email'")
    
    while (rs.next()) {
      println(s"id:${rs.getInt("id")}, name:${rs.getString("name")}")
    }

    conn.createStatement.executeUpdate("DROP TABLE IF EXISTS users")
    conn.close()
}